In [2]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
# import cv2
from unified_planning.shortcuts import *

In [4]:
#Claude Output
C_robot = [0,0]
C_P1 = [-6,8]
C_P2 = [-8,2]
C_P3 = [-2,4]
C_P4 = [4,6]
C_R1 = [6,0]
C_R2 = [-4,-7]
C_R3 = [7,-8]
C_G1 = [2,2]
C_G2 = [0,-4]


claude_x_points = [C_P1[0],C_P2[0],C_P3[0],C_P4[0],C_R1[0],C_R2[0],C_R3[0],C_G1[0],C_G2[0]]
claude_y_points = [C_P1[1],C_P2[1],C_P3[1],C_P4[1],C_R1[1],C_R2[1],C_R3[1],C_G1[1],C_G2[1]]

In [6]:
# Grid boxes
# Continuing with claude!

points = [C_robot,C_P1,C_P2,C_P3,C_P4,C_R1,C_R2,C_R3,C_G1,C_G2]


def normaliser(x,oldmin,oldmax,newmin,newmax):
    normalised = ((x - oldmin) / (oldmax - oldmin)) * (newmax - newmin) + newmin
    return normalised

final_locations = []
for point in points:
    x_norm = round(normaliser(point[0],-10,10,0,20))
    y_norm = round(normaliser(point[1],-10,10,0,20))
    final_locations.append([x_norm,y_norm])

# normalized_value = ((x - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min


In [7]:
#Unified Planning Object Locations

Location = UserType('Location')
Bricks = UserType('Bricks')
brick_at = unified_planning.model.Fluent('brick_at', BoolType(), l=Location)



In [8]:
#Create Grid Squares
locs = []
for y in range(20):
    for x in range(20):
        loc = f'loc-{x},{y}'
        locs.append(loc)
locations = [unified_planning.model.Object(l,Location) for l in locs]


In [9]:
#Create Brick Objects

seperate_bricks = ['P1','P2','P3','P4','R1','R2','R3','G1','G2']

bricks = [unified_planning.model.Object(b,Bricks) for b in seperate_bricks]




In [28]:
#States

brick_at = unified_planning.model.Fluent('brick_at', BoolType(), brick = Bricks, loc = Location)
on = unified_planning.model.Fluent('on', BoolType(), base_brick = Bricks, stack_brick = Bricks)
clear = unified_planning.model.Fluent('clear',BoolType(), brick = Bricks)



In [65]:
#Create Problem

problem = unified_planning.model.Problem('LegoScatter')

In [66]:
problem.add_fluent(brick_at, default_initial_value=False)
problem.add_fluent(on, default_initial_value=False)
problem.add_fluent(clear, default_initial_value=False)

bool clear[brick=Bricks]

In [67]:
#Move from one grid location to another grid square

move = unified_planning.model.InstantaneousAction('move',brick = Bricks, l_from=Location, l_to=Location)
curr_brick = move.parameter('brick')
l_from = move.parameter('l_from')
l_to = move.parameter('l_to')
move.add_precondition(brick_at(curr_brick, l_from))
move.add_precondition(clear(curr_brick))
move.add_effect(brick_at(curr_brick, l_from), False)
move.add_effect(brick_at(curr_brick, l_to), True)
problem.add_action(move)

In [68]:
#If two bricks are in the same location, stack one on top of the other

move_on = unified_planning.model.InstantaneousAction('move_on',base_brick = Bricks, stack_brick = Bricks, l_curr=Location)
base_brick = move_on.parameter('base_brick')
l_curr = move_on.parameter('l_curr')
stack_brick = move_on.parameter('stack_brick')
move_on.add_precondition(brick_at(base_brick, l_curr))
move_on.add_precondition(brick_at(stack_brick, l_curr))
move_on.add_precondition(clear(base_brick))
move_on.add_effect(on(stack_brick,base_brick ), True)
move_on.add_effect(clear(base_brick),False)
problem.add_action(move_on)

In [69]:
#Reversed - take a block off the top of the tower

move_off = unified_planning.model.InstantaneousAction('move_off',base_brick = Bricks, stack_brick = Bricks, l_curr=Location)
base_brick = move_off.parameter('base_brick')
l_curr = move_off.parameter('l_curr')
stack_brick = move_off.parameter('stack_brick')
move_off.add_precondition(brick_at(base_brick, l_curr))
move_off.add_precondition(brick_at(stack_brick, l_curr))
move_off.add_precondition(on(stack_brick,base_brick))
move_off.add_precondition(clear(stack_brick))
move_off.add_effect(on(stack_brick,base_brick ), False)
move_off.add_effect(clear(base_brick),True)
problem.add_action(move_off)

In [70]:
#Initialise Locations

for i in range(len(bricks)):
    loc = Object(f'loc-{final_locations[i+1][0]},{final_locations[i+1][1]}',Location)
    # print(loc)
    problem.set_initial_value(brick_at(bricks[i],loc),True)
    problem.set_initial_value(clear(bricks[i]), True)

In [71]:
print(problem)

problem name = LegoScatter

types = [Bricks, Location]

fluents = [
  bool brick_at[brick=Bricks, loc=Location]
  bool on[base_brick=Bricks, stack_brick=Bricks]
  bool clear[brick=Bricks]
]

actions = [
  action move(Bricks brick, Location l_from, Location l_to) {
    preconditions = [
      brick_at(brick, l_from)
      clear(brick)
    ]
    effects = [
      brick_at(brick, l_from) := false
      brick_at(brick, l_to) := true
    ]
  }
  action move_on(Bricks base_brick, Bricks stack_brick, Location l_curr) {
    preconditions = [
      brick_at(base_brick, l_curr)
      brick_at(stack_brick, l_curr)
      clear(base_brick)
    ]
    effects = [
      on(stack_brick, base_brick) := true
      clear(base_brick) := false
    ]
  }
  action move_off(Bricks base_brick, Bricks stack_brick, Location l_curr) {
    preconditions = [
      brick_at(base_brick, l_curr)
      brick_at(stack_brick, l_curr)
      on(stack_brick, base_brick)
      clear(stack_brick)
    ]
    effects = [
      on

In [72]:
#Hardcoded Goals
#Stack all corresponding colours
problem.add_goal(on(Object("P1",Bricks),Object("P2",Bricks)))
problem.add_goal(on(Object("P2",Bricks),Object("P3",Bricks)))
problem.add_goal(on(Object("P3",Bricks),Object("P4",Bricks)))
problem.add_goal(on(Object("R1",Bricks),Object("R2",Bricks)))
problem.add_goal(on(Object("R2",Bricks),Object("R3",Bricks)))
problem.add_goal(on(Object("G1",Bricks),Object("G2",Bricks)))

In [73]:
print(problem)

problem name = LegoScatter

types = [Bricks, Location]

fluents = [
  bool brick_at[brick=Bricks, loc=Location]
  bool on[base_brick=Bricks, stack_brick=Bricks]
  bool clear[brick=Bricks]
]

actions = [
  action move(Bricks brick, Location l_from, Location l_to) {
    preconditions = [
      brick_at(brick, l_from)
      clear(brick)
    ]
    effects = [
      brick_at(brick, l_from) := false
      brick_at(brick, l_to) := true
    ]
  }
  action move_on(Bricks base_brick, Bricks stack_brick, Location l_curr) {
    preconditions = [
      brick_at(base_brick, l_curr)
      brick_at(stack_brick, l_curr)
      clear(base_brick)
    ]
    effects = [
      on(stack_brick, base_brick) := true
      clear(base_brick) := false
    ]
  }
  action move_off(Bricks base_brick, Bricks stack_brick, Location l_curr) {
    preconditions = [
      brick_at(base_brick, l_curr)
      brick_at(stack_brick, l_curr)
      on(stack_brick, base_brick)
      clear(stack_brick)
    ]
    effects = [
      on